# Project Title
## Tourism Experience Analytics Using Regression, Classification and Recommendation System

## Github Link for this project :- https://github.com/shabbu8111999/Tourism_Experience_Analytics

## Problem Statement
### Tourism platforms generate large volumes of data related to user visits, attraction details, travel modes, and ratings. However, this data is often underutilized and stored across multiple datasets, making it difficult to extract meaningful insights and provide personalized experiences to users. The objective of this project is to analyze tourism-related data by integrating multiple datasets into a single consolidated dataset and applying machine learning techniques to:

### - Predict the rating a user is likely to give to a tourist attraction.

### - Classify the user’s visit mode (such as Business, Family, Couples, or Friends) based on historical travel behavior and demographic information.

### - Recommend tourist attractions to users based on their preferences and past interactions.

### By leveraging data cleaning, exploratory data analysis, and machine learning models, this project aims to help tourism businesses improve customer satisfaction, enable data-driven decision-making, and deliver personalized attraction recommendations.

## Importing Necessary Libraries

In [ ]:
# For System Operations
import os
import sys

# For Data Manipulation
import numpy as np
import pandas as pd

# For Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
sns.set(style="whitegrid")
plt.style.use("seaborn-v0_8")

# Statistical Analysis and Hypothesis Testing
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    StandardScaler
)

# Model Evaluation
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# Utilities
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Moving one level up to the project root folder
os.path.abspath('../')

## Dataset Loading and Initial Inspection

### Loading all the Datasets

In [12]:
# Loading Data
user_df = pd.read_excel('../data/User.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../data/User.xlsx'